In [13]:
### Import packages ###
import time
import numpy as np
import math as math
import pandas as pd
import random as random
from sklearn.cluster import AgglomerativeClustering

### Append Path ###
import sys
sys.path.append('..')

### Import functions ###
from utils.Main import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Prediction import *

# Inputs

In [14]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, "..",".."))

### DataType ###
DataFileInput = "FICO"  # BankNote  Bar7  BreastCancer  CarEvaluation  COMPAS  FICO  Haberman  Iris  MONK1  MONK3

In [15]:
### Parameter Vector ###
# ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "ParameterVectors", "ParameterVector" + DataFileInput + ".csv"))

# ### Parameter Vector ###
# for SimV in range(0,len(ParameterVector)):
#     SimulationConfigInput = {"DataFileInput": ParameterVector.iloc[SimV]["Data"],
#                             "Seed": int(ParameterVector.iloc[SimV]["Seed"]),
#                             "TestProportion": float(ParameterVector.iloc[SimV]["TestProportion"]),
#                             "CandidateProportion": float(ParameterVector.iloc[SimV]["CandidateProportion"]),
#                             "SelectorType": str(ParameterVector.iloc[SimV]["SelectorType"]), 
#                             "ModelType": str(ParameterVector.iloc[SimV]["ModelType"]), 
#                             "UniqueErrorsInput": int(ParameterVector.iloc[SimV]["UniqueErrorsInput"]),
#                             "n_estimators":int(ParameterVector.iloc[SimV]["n_estimators"]),
#                             "regularization":float(ParameterVector.iloc[SimV]["regularization"]),
#                             "RashomonThresholdType":ParameterVector.iloc[SimV]["RashomonThresholdType"],
#                             "RashomonThreshold":float(ParameterVector.iloc[SimV]["RashomonThreshold"]),
#                             "Type":ParameterVector.iloc[SimV]["Type"]
#                             }

SimulationConfigInput = {'DataFileInput': DataFileInput,
    'Seed': 1,
    'TestProportion': 0.2,
    'CandidateProportion': 0.8,
    'SelectorType': 'BatchQBCDiversityFunction', # BatchQBCDiversityFunction, PassiveLearning
    'ModelType': 'RandomForestClassificationFunction', # RandomForestClassificationFunction, TreeFarmsFunction
    'UniqueErrorsInput': 0,
    'n_estimators': 100,
    'regularization': 0.01,
    'RashomonThresholdType': "Adder",
    'RashomonThreshold': 0.005,
    'Type': 'Classification',
    "DiversityWeight" : 0.4,
    "BatchSize" : 10}


In [16]:
### Set Up ###
StartTime = time.time()
random.seed(SimulationConfigInput["Seed"])
np.random.seed(SimulationConfigInput["Seed"])

### Load Data ###
df = LoadData(SimulationConfigInput["DataFileInput"])

### Train Test Candidate Split ###
from utils.Main import TrainTestCandidateSplit                           ### NOTE: Why is this not imported from utils.Main import *
df_Train, df_Test, df_Candidate = TrainTestCandidateSplit(df, SimulationConfigInput["TestProportion"], SimulationConfigInput["CandidateProportion"])

### Batch Active Learning Metrics ###
# Set Up #
X_Candidate = df_Candidate.loc[:, df_Candidate.columns!= "Y"]
X_Train = df_Train.loc[:,df_Train.columns!= "Y"]

# Clustering #
cluster = AgglomerativeClustering(n_clusters=5, linkage="average")
ClusterLabels = cluster.fit_predict(X_Candidate)
df_Candidate["ClusterLabels"] = ClusterLabels

# Diversity Metric #
d_nmX = cdist(X_Candidate, X_Train, metric = "euclidean")
d_nX = d_nmX.min(axis=1)
df_Candidate["d_nX"] = d_nX

### Update SimulationConfig Arguments ###
SimulationConfigInput['df_Train'] = df_Train
SimulationConfigInput["df_Test"] = df_Test
SimulationConfigInput["df_Candidate"] = df_Candidate

# Learning Procedure

In [17]:
### Input ###
SimulationConfigInputUpdated = SimulationConfigInput

---

In [12]:
Results = OneIterationFunction(SimulationConfigInputUpdated)

Iteration: 0
df_Candidate obs: 4420
Iteration: 1
df_Candidate obs: 4410
Iteration: 2
df_Candidate obs: 4400
Iteration: 3
df_Candidate obs: 4390
Iteration: 4
df_Candidate obs: 4380
Iteration: 5
df_Candidate obs: 4370
Iteration: 6
df_Candidate obs: 4360
Iteration: 7
df_Candidate obs: 4350
Iteration: 8


KeyboardInterrupt: 

---

In [ ]:
### Set Up ###
ErrorVec = []
SelectedObservationHistory = []
TreeCount = {"AllTreeCount": [], "UniqueTreeCount": []}

In [ ]:
i=1

In [ ]:
### Prediction Model ###
print("Iteration: " + str(i))
ModelType = globals().get(SimulationConfigInputUpdated["ModelType"], None)
ModelArgsFiltered = FilterArguments(ModelType, SimulationConfigInputUpdated)
Model = ModelType(**ModelArgsFiltered)
SimulationConfigInputUpdated['Model'] = Model


In [ ]:
### Test Error ###
TestErrorOutput = TestErrorFunction(InputModel = Model, df_Test = SimulationConfigInputUpdated["df_Test"], Type = SimulationConfigInputUpdated["Type"])
if('TREEFARMS' in str(type(Model))):                                                       # If Rashomon
    CurrentError = TestErrorOutput["Error_Duplicate"]
else: 
    CurrentError = TestErrorOutput["ErrorVal"]                                               # One output for non-Rashomon
ErrorVec.append(CurrentError)


In [ ]:
### Sampling Procedure ###
SelectorType = globals().get(SimulationConfigInputUpdated["SelectorType"], None)
SelectorArgsFiltered = FilterArguments(SelectorType, SimulationConfigInputUpdated)
SelectorFuncOutput = SelectorType(**SelectorArgsFiltered)
QueryObservationIndex = SelectorFuncOutput["IndexRecommendation"]
QueryObservation = SimulationConfigInputUpdated["df_Candidate"].loc[QueryObservationIndex]
SelectedObservationHistory.append(QueryObservationIndex)


In [ ]:
### Update Train and Candidate Sets ###
SimulationConfigInputUpdated["df_Train"] = pd.concat([SimulationConfigInputUpdated["df_Train"], QueryObservation]).drop(columns=['ClusterLabels', 'd_nX'])
SimulationConfigInputUpdated["df_Candidate"] = SimulationConfigInputUpdated["df_Candidate"].drop(QueryObservationIndex) 

In [ ]:
### Store Number of (Unique) Trees ###
if('TREEFARMS' in str(type(Model))):
    TreeCount["AllTreeCount"].append(SelectorFuncOutput["AllTreeCount"])          # Store number of trees
    TreeCount["UniqueTreeCount"].append(SelectorFuncOutput["UniqueTreeCount"])    # Store number of unique/duplicate trees


# Save

In [ ]:
### Return Simulation Parameters ###
SimulationParameters = {"DataFileInput" : str(SimulationConfigInput["DataFileInput"]),
                            "Seed" : str(SimulationConfigInput["Seed"]),
                            "TestProportion" : str(SimulationConfigInput["TestProportion"]),
                            "CandidateProportion" : str(SimulationConfigInput["CandidateProportion"]),
                            "SelectorType" :  str(SimulationConfigInput["SelectorType"]),
                            "ModelType" :  str(SimulationConfigInput["ModelType"]),
                            'UniqueErrorsInput': str(SimulationConfigInput["UniqueErrorsInput"]),
                            'n_estimators': str(SimulationConfigInput["n_estimators"]),
                            'regularization': str(SimulationConfigInput["regularization"]),
                            'RashomonThreshold': str(SimulationConfigInput["RashomonThreshold"]),
                            'Type': 'Classification',
                            }

In [ ]:
### Return Time ###
ElapsedTime = time.time() - StartTime

### Return Dictionary ###
SimulationResults = {"ErrorVec" : pd.DataFrame(ErrorVec, columns =["Error"]),
                            "SelectionHistory" : pd.DataFrame(SelectedObservationHistory, columns = ["ObservationID"]),
                            "SimulationParameters" : SimulationParameters,
                            "ElapsedTime" : ElapsedTime}